## Deep Learning Prototype

## Imports

In [22]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Load Training CSV File

Displays dataframe for verification purposes.

In [18]:
# Load the training CSV file
csv_file = 'training_labels.csv'
training_df = pd.read_csv(csv_file)

image_directory = 'vertical_speed_frames'
training_df['image_path'] = training_df['image_path'].apply(lambda x: os.path.join(image_directory, x))

training_df


,image_path,vertical_speed
0,frames/frame_0000.jpg,-6.00
1,frames/frame_0001.jpg,-6.00
2,frames/frame_0002.jpg,-6.00
3,frames/frame_0003.jpg,-6.00
4,frames/frame_0004.jpg,-6.00
...,...,...
695,frames/frame_0695.jpg,1.75
696,frames/frame_0696.jpg,1.75
697,frames/frame_0697.jpg,1.75
698,frames/frame_0698.jpg,1.75


## Helper Function - Load Image

In [ ]:
def load_image(file_path):
    image = cv2.imread(file_path)

    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

## Load Images and apply labels from CSV

In [19]:
# Load images into a NumPy array
images = np.array([load_image(file_path) for file_path in training_df['image_path']])


# Encode the labels (assuming 'vertical_speed' is the column containing labels)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(training_df['vertical_speed'])

# Optionally, one-hot encode the labels (if using categorical cross-entropy)
num_classes = len(label_encoder.classes_)
one_hot_labels = tf.keras.utils.to_categorical(encoded_labels, num_classes)


# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, one_hot_labels, test_size=0.2, random_state=42)

## Execute - Run Training Model

In [25]:
# Define the model architecture
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(120, 125, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')  # Adjust num_classes as needed
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use appropriate loss function
              metrics=['accuracy'])

# Load and preprocess your dataset
# You can use TensorFlow's `ImageDataGenerator` for data augmentation and preprocessing.

# Train the model
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val))

# Save the trained model
model.save('vertical_speed_model.h5')


Epoch 1/20
18/18 [==============================] - 1s 74ms/step - loss: 3.5910 - accuracy: 0.1411 - val_loss: 3.4058 - val_accuracy: 0.1857
Epoch 2/20
18/18 [==============================] - 1s 69ms/step - loss: 2.9249 - accuracy: 0.2750 - val_loss: 2.6544 - val_accuracy: 0.3643
Epoch 3/20
18/18 [==============================] - 1s 70ms/step - loss: 1.9625 - accuracy: 0.5071 - val_loss: 1.6937 - val_accuracy: 0.5643
Epoch 4/20
18/18 [==============================] - 1s 74ms/step - loss: 1.0653 - accuracy: 0.6821 - val_loss: 1.0904 - val_accuracy: 0.7071
Epoch 5/20
18/18 [==============================] - 1s 70ms/step - loss: 0.6550 - accuracy: 0.8125 - val_loss: 0.8608 - val_accuracy: 0.7929
Epoch 6/20
18/18 [==============================] - 1s 70ms/step - loss: 0.5672 - accuracy: 0.8214 - val_loss: 0.7057 - val_accuracy: 0.7714
Epoch 7/20
18/18 [==============================] - 1s 70ms/step - loss: 0.4394 - accuracy: 0.8482 - val_loss: 0.5953 - val_accuracy: 0.8429
Epoch 8/20
18

/opt/homebrew/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
